Используем sqlalchemy для коннекта к БД

In [1]:
from sqlalchemy import create_engine
import pandas as pd

In [2]:
engine = create_engine('postgresql://postgres@127.0.0.1:5432')

Читаем данные в pandas из postgresql

In [3]:
bankacc = pd.read_sql_query('select * from bankacc',con=engine)
collateral = pd.read_sql_query('select * from collateral',con=engine)
positions = pd.read_sql_query('select * from positions',con=engine)
market_risk = pd.read_sql_query('select * from market_risk',con=engine)

Получаем расчетные кода, допустившие технический овердрафт
select
  *
from collateral
where volume < 0
;

In [5]:
collateral.loc[collateral['volume'] < 0]

,bankaccid,currencyid,volume
27,21098,EUR,-511000.00
31,43210,RUB,-776.88
51,54321,RUB,-50.00


Узнаем тип расчетных кодов, допустивших технический овердрафт, и к каким фирмам они принадлежат
select
  *
from collateral c
join bankacc b on b.bankaccid = c.bankaccid
where c.volume < 0
;

In [6]:
collateral.loc[collateral['volume'] < 0].merge(bankacc, how='inner', left_on='bankaccid', right_on='bankaccid')

,bankaccid,currencyid,volume,typ,firmid
0,21098,EUR,-511000.00,S,XZ7413500000
1,43210,RUB,-776.88,S,XZ3115900000
2,54321,RUB,-50.00,L,XZ9853400000


Получаем суммарную позицию в разрезе расчетный код/актив
select
  bankaccid,
  currencyid,
  sum(volume) as volume
from positions
group by bankaccid, currencyid
;

In [7]:
positions.groupby(['bankaccid', 'currencyid'])['volume'].sum()

bankaccid  currencyid
01234      CNY          -1.481000e+08
           EUR           1.126160e+08
           RUB           1.124301e+10
           USD          -2.977625e+08
10987      EUR           1.070000e+05
           RUB          -8.584839e+08
           USD           1.382600e+07
12345      EUR          -6.333000e+06
           RUB          -3.823992e+08
           USD           1.761234e+07
21098      CNY          -2.000000e+06
           EUR          -5.513700e+07
           GBP           5.200000e+06
           RUB           1.595995e+11
           USD          -2.536568e+09
23456      CNY          -5.700000e+06
           RUB           4.486065e+07
           USD           1.720000e+05
32109      EUR           1.707100e+04
           RUB           4.939018e+06
           USD          -9.985648e+04
34567      EUR          -1.000000e+05
           RUB          -1.909845e+06
           USD           1.530000e+05
36985      RUB          -8.150000e+04
43210      CNY           6.3

 Получаем суммарную позицию в разрезе фирма/день/актив
select
  b.firmid,
  p.settledate,
  p.currencyid,
  sum(p.volume)
from positions p
join bankacc b on b.bankaccid = p.bankaccid
group by b.firmid, p.settledate, p.currencyid
order by b.firmid, p.settledate
;

In [14]:
positions.merge(bankacc, how='inner', left_on='bankaccid', right_on='bankaccid') \
.groupby(['firmid', 'settledate', 'currencyid'])['volume'].sum()

firmid        settledate  currencyid
XZ3115900000  2018-05-29  CNY           6.323000e+06
                          EUR          -4.516980e+08
                          GLD          -1.990000e+03
                          RUB           3.142174e+10
                          USD           2.000130e+07
              2018-06-19  RUB           6.225930e+07
                          USD          -1.000000e+06
              2018-06-21  RUB          -3.736392e+09
                          USD           6.000000e+07
              2018-06-26  RUB           3.115410e+09
                          USD          -5.000000e+07
              2018-07-05  RUB          -1.559270e+09
                          USD           2.500000e+07
              2018-07-11  RUB           6.241240e+07
                          USD          -1.000000e+06
              2018-07-13  RUB           1.623073e+09
                          USD          -2.600000e+07
              2018-07-16  RUB           3.122325e+08
         

Получем переоценку залоговых средств по текущей цене
select
  b.firmid,
  sum(c.volume * r.price_clean) as value
from collateral c
join (
  select distinct
    asset,
    price_clean
  from market_risk
) r on c.currencyid = r.asset
join bankacc b on b.bankaccid = c.bankaccid
group by b.firmid
;

In [27]:
csv = collateral.merge(market_risk[['asset', 'price_clean']].drop_duplicates(), how='inner', left_on='currencyid', right_on='asset')\
.merge(bankacc, how='inner', left_on='bankaccid', right_on='bankaccid') \
.assign(value=lambda idf: idf['volume']*idf['price_clean'])\
.groupby(['firmid'])['value'].sum()
csv

firmid
XZ3115900000    8.045005e+09
XZ3644700000    5.107635e+08
XZ6987400000    2.582551e+09
XZ7413500000    2.981411e+08
XZ9853400000    6.649487e+10
Name: value, dtype: float64

In [28]:
csv.to_csv('.\data\out.csv')